# muss man einbinden

Importieren

In [5]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import plotly.express as px
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

einlesen und Trainings- und Testdaten bestimmen

In [6]:
# CSV-Datei einlesen
average_values = pd.read_csv("werte_daten/average_values.csv")
# es werden nur die nötigten Werte genommen
land_PM2_5_average = average_values[["datum", "average_landpm2_5"]]
# Das Datum als Index setzen
land_PM2_5_average.set_index('datum', inplace=True)

In [7]:
# ein Series mit den Durchschnittsdaten erstellen
values_lpm2_5 = land_PM2_5_average["average_landpm2_5"]
# das filtern der Datumswerte für die Visualisierung später
zeitreihen_index_lpm2_5 = values_lpm2_5.index

In [8]:
# es wird eine Definition erstellt, die ein Dataframe in X und y aufteilt, mit Betrachtung eines Windows
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+5]]
        X.append(row)
        label = df_as_np[i+5]
        y.append(label)
    return np.array(X), np.array(y)

In [9]:
# X und y werden erstellt und die Größe ausgegeben
WINDOW_SIZE = 5
X, y = df_to_X_y(values_lpm2_5, WINDOW_SIZE)
X.shape, y.shape

((369, 5, 1), (369,))

In [10]:
# die Werte werden unterteilt
X_train, y_train = X[:280], y[:280]
train_index = zeitreihen_index_lpm2_5[5:285]
X_val, y_val = X[:300], y[:300]
val_index = zeitreihen_index_lpm2_5[5:305]
X_test, y_test = X[300:], y[300:]
test_index = zeitreihen_index_lpm2_5[305:]

Model bauen und trainieren

In [214]:
# das Model wird erstellt
model1 = Sequential()
model1.add(InputLayer((5,1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

In [115]:
# es wird ein Checkpoint erstellt, sodass am Ende das beste Model gespeichert wird
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])

Achtung, falls die untere Zeile abbricht, einfach nochmal ausführen, keine Ahnung warum das passiert, muss nicht unbedingt ausgeführt werden. Siehe nächste Markdown-Zelle

In [116]:
# das Modell wird trainiert
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
model1.save('model1.keras')

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 24.9671 - root_mean_squared_error: 4.9943 - val_loss: 20.3910 - val_root_mean_squared_error: 4.5156
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 14.7687 - root_mean_squared_error: 3.8277 - val_loss: 15.2547 - val_root_mean_squared_error: 3.9057
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 15.5400 - root_mean_squared_error: 3.9285 - val_loss: 12.1870 - val_root_mean_squared_error: 3.4910
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 11.3888 - root_mean_squared_error: 3.3704 - val_loss: 11.8312 - val_root_mean_squared_error: 3.4397
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 13.9849 - root_mean_squared_error: 3.7127 - val_loss: 11.3385 - val_root_mean_squared_error: 3.3673
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 8.8220 - root_mean_squared_error: 2.9491 - val_loss: 11.0343 - val_root_mean_squared_error: 3.3218
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - l

das load wird verwendet, sodass immer das gleiche Model genutzt werden kann

In [11]:
model1 = load_model('model1.keras')

darstellen von Vergleichen durch visuelle Darstellung

In [12]:
# es werden die Traingsdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
train_predictions = model1.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions' : train_predictions, 'Actuals' : y_train}, index=train_index)
from sklearn.metrics import mean_absolute_error
# Berechnung des Mean Absolute Error (MAE) und R^2 und des MSE
mae_lpm2_5= mean_absolute_error(train_results['Actuals'], train_results['Train Predictions'])
r_squared_lpm2_5 = r2_score(train_results['Actuals'], train_results['Train Predictions'])
mse_lpm2_5 = mean_squared_error(train_results['Actuals'], train_results['Train Predictions'])

print(f'Mean Absolute Error: {mae_lpm2_5}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Mean Absolute Error: 1.8387179004703251


In [13]:
# Plotly Express zur Visualisierung verwenden
fig = px.line(train_results, x=train_results.index, y=['Train Predictions', 'Actuals'], title='Train Predictions vs Actuals')
fig.show()

In [14]:
# es werden die Validierungsdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
val_predictions = model1.predict(X_val).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val}, index=val_index)
val_results

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,Val Predictions,Actuals
datum,,
2022-04-17,4.001770,1.804000
2022-04-19,2.283633,2.157500
2022-04-21,2.472489,8.485000
2022-04-23,4.731039,6.510000
2022-04-25,4.514254,7.360000
...,...,...
2023-12-11,8.710408,1.433333
2023-12-13,3.969244,3.450000
2023-12-15,6.248790,8.955000


In [15]:
# Plotly Express zur Visualisierung verwenden
fig = px.line(val_results, x=val_results.index, y=['Val Predictions', 'Actuals'], title='Val Predictions vs Actuals')
fig.show()

In [16]:
# es werden die Testdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
test_predictions = model1.predict(X_test).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test}, index=test_index)
test_results

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


,Test Predictions,Actuals
datum,,
2023-12-21,2.783000,0.5750
2023-12-23,2.510494,1.0000
2023-12-25,2.779042,1.2925
2023-12-27,2.318035,2.5000
2023-12-29,2.751903,1.7875
...,...,...
2024-05-21,2.167176,1.7150
2024-05-23,2.314510,1.9000
2024-05-25,2.317093,1.4700


In [17]:
# Plotly Express zur Visualisierung verwenden
fig = px.line(test_results, x=test_results.index, y=['Test Predictions', 'Actuals'], title='Test Predictions vs Actuals')
fig.show()

nun werden Zukunfsvorhersagen gemacht und visuell dargestellt

In [18]:
# es wird eine Funktion erstellt, um Vorhersagen für die Zukunft zu machen
def update_X_next(X_prev, y_pred):
    X_next = np.append(X_prev[:, 1:, :], np.expand_dims(y_pred, axis=1), axis=1)
    return X_next

In [19]:
# # Start mit den initialen Eingabedaten
X_next = X_test  
predictions = []

for _ in range(5):
    y_pred = model1.predict(X_next)
    predictions.append(y_pred)
    # Aktualisieren von X_next für den nächsten Schritt
    X_next = update_X_next(X_next, y_pred)  

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step


In [20]:
# es werden neue Daten für die Zukunft erstellt
jahre = ["2024", "2025", "2026"]
monate = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
tage = ["01", "03", "05", "07", "09", "11", "13", "15", "17", "19", "21", "23", "25", "27", "29"]

start_datum = '2024-06-01'

# Erzeugen eines neuen Index basierend auf den angegebenen Listen
new_index = []
for jahr in jahre:
    for monat in monate:
        for tag in tage:
            date_str = f"{jahr}-{monat}-{tag}"
            if date_str >= start_datum:
                new_index.append(date_str)
                if len(new_index) == 80:
                    break
        else:
            continue
        break
    else:
        continue
    break

In [21]:
# Konvertiere new_index in einen DatetimeIndex
new_index_dt = pd.to_datetime(new_index)

# Kombiniere beide Indexstrukturen
gesamt_index = zeitreihen_index_lpm2_5.append(new_index_dt)

In [22]:
# Vorhersage für die Zukunft
y_pred_step1 = model1.predict(X_test)

# Vorbereiten der Daten für Plotly Express
historical_data = values_lpm2_5

# Plotly Express-Figur erstellen
fig = px.line(title='Vorhersage für den ersten Schritt mit LSTM-Modell')

# Plot der historischen Daten
fig.add_scatter(x=gesamt_index[:len(historical_data)], y=historical_data, mode='lines+markers', name='Historische Daten')

# Plot der Vorhersage für den ersten Schritt
x_values_pred = gesamt_index[len(historical_data):len(historical_data) + len(y_pred_step1)]
fig.add_scatter(x=x_values_pred, y=y_pred_step1.flatten(), mode='lines+markers', name='Vorhersage')

fig.update_layout(xaxis_title='Zeit', yaxis_title='Wert', title='Vorhersage für den Wert PM2.5 auf dem Land mit LSTM-Modell')
fig.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


nun wird die Regressionsgerade berechnet

In [217]:
# Datum in numerisches Format umwandeln
land_PM2_5_average = land_PM2_5_average.copy()
land_PM2_5_average.reset_index(inplace=True)
land_PM2_5_average['datum'] = pd.to_datetime(land_PM2_5_average['datum'])
land_PM2_5_average['datum_numeric'] = (land_PM2_5_average['datum'] - land_PM2_5_average['datum'].min()).dt.days

# Lineare Regression
X = land_PM2_5_average[['datum_numeric']]
y = land_PM2_5_average['average_landpm2_5']
model = LinearRegression()
model.fit(X, y)
land_PM2_5_average['land_PM2_5_average_trend'] = model.predict(X)

In [218]:
# Berechnung von R^2 und MSE und MAE und die Steigung
r_squared_lpm2_5_regression = r2_score(y, land_PM2_5_average['land_PM2_5_average_trend'])
mse_lpm2_5_regression = mean_squared_error(y, land_PM2_5_average['land_PM2_5_average_trend'])
mae_lpm2_5_regression = mean_absolute_error(y, land_PM2_5_average['land_PM2_5_average_trend'])
slope_lpm2_5 = model.coef_[0]

# Visualisierung mit Plotly Express
fig = px.scatter(land_PM2_5_average, x='datum', y='average_landpm2_5', title='Lineare Regression für average_landpm2_5')
fig.add_scatter(x=land_PM2_5_average['datum'], y=land_PM2_5_average['land_PM2_5_average_trend'], mode='lines', name='Trend')
fig.update_layout(xaxis_title='Datum', yaxis_title='average_landpm2_5')
fig.show()

hier fängt die zweite Tabelle an, also die Werte Land PM10

es wird wieder die Tabelle eingelesen und in Trainings- und Testdaten unterteilt

In [220]:
# CSV-Datei einlesen
average_values = pd.read_csv("werte_daten/average_values.csv")
# es werden nur die nötigten Werte genommen
land_PM10_average = average_values[["datum", "average_landpm10"]]
# Das Datum als Index setzen
land_PM10_average.set_index('datum', inplace=True)

# ein Series mit den Durchschnittsdaten erstellen
values_lpm10 = land_PM10_average["average_landpm10"]
# das filtern der Datumswerte für die Visualisierung später
zeitreihen_index_lpm10 = values_lpm10.index

In [221]:
# die Funktion ist dafür da, um die Values in X und y Werte zu unterteilen, mit einer gegebenen window-size
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+5]]
        X.append(row)
        label = df_as_np[i+5]
        y.append(label)
    return np.array(X), np.array(y)

WINDOW_SIZE = 5
X, y = df_to_X_y(values_lpm10, WINDOW_SIZE)
X.shape, y.shape

((369, 5, 1), (369,))

In [222]:
# die Daten werden in Traings-, Testdaten unterteilt, zudem wird auch der Index unterteilt
X_train, y_train = X[:280], y[:280]
train_index = zeitreihen_index_lpm10[5:285]
X_val, y_val = X[:300], y[:300]
val_index = zeitreihen_index_lpm10[5:305]
X_test, y_test = X[300:], y[300:]
test_index = zeitreihen_index_lpm10[305:]

es wird wieder das Modell trainiert

In [102]:
# es wird ein Modell erstellt, trainiert und gespeichert
model2 = Sequential()
model2.add(InputLayer((5,1)))
model2.add(LSTM(64))
model2.add(Dense(8, 'relu'))
model2.add(Dense(1, 'linear'))
model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
model2.save('model2.keras')

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 77.5394 - root_mean_squared_error: 8.7720 - val_loss: 56.3892 - val_root_mean_squared_error: 7.5093
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 54.7351 - root_mean_squared_error: 7.3612 - val_loss: 42.5005 - val_root_mean_squared_error: 6.5192
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 45.9476 - root_mean_squared_error: 6.7434 - val_loss: 33.5287 - val_root_mean_squared_error: 5.7904
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 25.6975 - root_mean_squared_error: 5.0390 - val_loss: 30.4349 - val_root_mean_squared_error: 5.5168
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 34.0884 - root_mean_squared_error: 5.8141 - val_loss: 29.5652 - val_root_mean_squared_error: 5.4374
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 28.8410 - root_mean_squared_error: 5.3468 - val_loss: 28.4604 - val_root_mean_squared_error: 5.3348
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - l

In [223]:
# das Laden des Modells
model2 = load_model('model2.keras')

In [225]:
# es werden die Traingsdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
train_predictions = model2.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions' : train_predictions, 'Actuals' : y_train}, index=train_index)
from sklearn.metrics import mean_absolute_error
# Berechnung des Mean Absolute Error (MAE) und R^2 und des MSE
mae_lpm10= mean_absolute_error(train_results['Actuals'], train_results['Train Predictions'])
r_squared_lpm10 = r2_score(train_results['Actuals'], train_results['Train Predictions'])
mse_lpm10 = mean_squared_error(train_results['Actuals'], train_results['Train Predictions'])

print(f'Mean Absolute Error: {mae_lpm10}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Mean Absolute Error: 2.6908866172472634


es werden wieder Zukunftsvorhersagen gemacht

In [105]:
# die Eingabedaten für die Vorhersage der Zukunft werden erstellt
X_next = X_test
predictions = []

for _ in range(5):
    y_pred = model1.predict(X_next)
    predictions.append(y_pred)
    # Aktualisieren von X_next für den nächsten Schritt
    X_next = update_X_next(X_next, y_pred)  

# Konvertiere new_index in einen DatetimeIndex
new_index_dt = pd.to_datetime(new_index)

# Kombiniere beide Indexstrukturen
gesamt_index = zeitreihen_index_lpm10.append(new_index_dt)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [106]:
# Vorhersage für den ersten Schritt machen
y_pred_step1 = model2.predict(X_test)

# Vorbereiten der Daten für Plotly Express
historical_data = values_lpm10

# Plotly Express-Figur erstellen
fig = px.line(title='Vorhersage für den ersten Schritt mit LSTM-Modell')

# Plot der historischen Daten
fig.add_scatter(x=gesamt_index[:len(historical_data)], y=historical_data, mode='lines+markers', name='Historische Daten')

# Plot der Vorhersage für den ersten Schritt
x_values_pred = gesamt_index[len(historical_data):len(historical_data) + len(y_pred_step1)]
fig.add_scatter(x=x_values_pred, y=y_pred_step1.flatten(), mode='lines+markers', name='Vorhersage')

fig.update_layout(xaxis_title='Zeit', yaxis_title='Wert', title='Vorhersage für den Wert PM10 auf dem Land mit LSTM-Modell')
fig.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


nun wird die Regressionsgerade berechnet

In [226]:
# Datum in numerisches Format umwandeln
land_PM10_average = land_PM10_average.copy()
land_PM10_average.reset_index(inplace=True)
land_PM10_average['datum'] = pd.to_datetime(land_PM10_average['datum'])
land_PM10_average['datum_numeric'] = (land_PM10_average['datum'] - land_PM10_average['datum'].min()).dt.days

# Lineare Regression
X = land_PM10_average[['datum_numeric']]
y = land_PM10_average['average_landpm10']
model = LinearRegression()
model.fit(X, y)
land_PM10_average['land_PM10_average_trend'] = model.predict(X)

In [229]:
print(r_squared_lpm10_regression)
print(mae_lpm10_regression)
print(slope_lpm10)

0.0010528058118836059
3.9292037094894616
-0.0008877413879296886


In [227]:
# Berechnung von R^2 und MSE und MAE und die Steigung
r_squared_lpm10_regression = r2_score(y, land_PM10_average['land_PM10_average_trend'])
mse_lpm10_regression = mean_squared_error(y, land_PM10_average['land_PM10_average_trend'])
mae_lpm10_regression = mean_absolute_error(y, land_PM10_average['land_PM10_average_trend'])
slope_lpm10 = model.coef_[0]

# Visualisierung mit Plotly Express
fig = px.scatter(land_PM10_average, x='datum', y='average_landpm10', title='Lineare Regression für average_landpm10')
fig.add_scatter(x=land_PM10_average['datum'], y=land_PM10_average['land_PM10_average_trend'], mode='lines', name='Trend')
fig.update_layout(xaxis_title='Datum', yaxis_title='average_landpm10')
fig.show()

Jetzt wird die Prognose für die Tabelle der PM2.5 Werte in der Stadt gemacht

es wird wieder die Tabelle eingelesen und in Trainings- und Testdaten unterteilt

In [230]:
# CSV-Datei einlesen
average_values = pd.read_csv("werte_daten/average_values.csv")
# es werden nur die nötigten Werte genommen
stadt_PM2_5_average = average_values[["datum", "average_stadtpm2_5"]]
# Das Datum als Index setzen
stadt_PM2_5_average.set_index('datum', inplace=True)

# ein Series mit den Durchschnittsdaten erstellen
values_spm2_5 = stadt_PM2_5_average["average_stadtpm2_5"]
# das filtern der Datumswerte für die Visualisierung später
zeitreihen_index_spm2_5 = values_spm2_5.index

In [231]:
# die Funktion ist dafür da, um die Values in X und y Werte zu unterteilen, mit einer gegebenen window-size
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+5]]
        X.append(row)
        label = df_as_np[i+5]
        y.append(label)
    return np.array(X), np.array(y)

WINDOW_SIZE = 5
X, y = df_to_X_y(values_spm2_5, WINDOW_SIZE)
X.shape, y.shape

((369, 5, 1), (369,))

In [232]:
# die Daten werden in Traings-, Testdaten unterteilt, zudem wird auch der Index unterteilt
X_train, y_train = X[:280], y[:280]
train_index = zeitreihen_index_spm2_5[5:285]
X_val, y_val = X[:300], y[:300]
val_index = zeitreihen_index_spm2_5[5:305]
X_test, y_test = X[300:], y[300:]
test_index = zeitreihen_index_spm2_5[305:]

es wird wieder das Modell trainiert

In [135]:
# es wird ein Modell erstellt, trainiert und gespeichert
model3 = Sequential()
model3.add(InputLayer((5,1)))
model3.add(LSTM(64))
model3.add(Dense(8, 'relu'))
model3.add(Dense(1, 'linear'))
model3.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
model3.save('model3.keras')

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - loss: 44.8720 - root_mean_squared_error: 6.6788 - val_loss: 44.1817 - val_root_mean_squared_error: 6.6469
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 47.5184 - root_mean_squared_error: 6.8297 - val_loss: 36.6400 - val_root_mean_squared_error: 6.0531
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 32.5161 - root_mean_squared_error: 5.7006 - val_loss: 27.6277 - val_root_mean_squared_error: 5.2562
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 21.8416 - root_mean_squared_error: 4.6679 - val_loss: 22.5487 - val_root_mean_squared_error: 4.7485
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 22.1186 - root_mean_squared_error: 4.6942 - val_loss: 21.6734 - val_root_mean_squared_error: 4.6555
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 20.5259 - root_mean_squared_error: 4.5165 - val_loss: 21.3824 - val_root_mean_squared_error: 4.6241
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - 

In [233]:
# das Laden des Modells
model3 = load_model('model3.keras')

In [234]:
# es werden die Traingsdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
train_predictions = model3.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions' : train_predictions, 'Actuals' : y_train}, index=train_index)
from sklearn.metrics import mean_absolute_error
# Berechnung des Mean Absolute Error (MAE) und R^2 und des MSE
mae_spm2_5 = mean_absolute_error(train_results['Actuals'], train_results['Train Predictions'])
r_squared_spm2_5 = r2_score(train_results['Actuals'], train_results['Train Predictions'])
mse_spm2_5 = mean_squared_error(train_results['Actuals'], train_results['Train Predictions'])

print(f'Mean Absolute Error: {mae_spm2_5}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Mean Absolute Error: 2.6354170004163464


es werden wieder Zukunftsvorhersagen gemacht

In [138]:
# die Eingabedaten für die Vorhersage der Zukunft werden erstellt
X_next = X_test
predictions = []

for _ in range(5):
    y_pred = model1.predict(X_next)
    predictions.append(y_pred)
    # Aktualisieren von X_next für den nächsten Schritt
    X_next = update_X_next(X_next, y_pred)  

# Konvertiere new_index in einen DatetimeIndex
new_index_dt = pd.to_datetime(new_index)

# Kombiniere beide Indexstrukturen
gesamt_index = zeitreihen_index_spm2_5.append(new_index_dt)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  


In [139]:
# Vorhersage für den ersten Schritt machen
y_pred_step1 = model3.predict(X_test)

# Vorbereiten der Daten für Plotly Express
historical_data = values_spm2_5

# Plotly Express-Figur erstellen
fig = px.line(title='Vorhersage für den ersten Schritt mit LSTM-Modell')

# Plot der historischen Daten
fig.add_scatter(x=gesamt_index[:len(historical_data)], y=historical_data, mode='lines+markers', name='Historische Daten')

# Plot der Vorhersage für den ersten Schritt
x_values_pred = gesamt_index[len(historical_data):len(historical_data) + len(y_pred_step1)]
fig.add_scatter(x=x_values_pred, y=y_pred_step1.flatten(), mode='lines+markers', name='Vorhersage')

fig.update_layout(xaxis_title='Zeit', yaxis_title='Wert', title='Vorhersage für den Wert PM2.5 in der Stadt mit LSTM-Modell')
fig.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


Nun wird die Regressionsgerade berechnet

In [235]:
# Datum in numerisches Format umwandeln
stadt_PM2_5_average = stadt_PM2_5_average.copy()
stadt_PM2_5_average.reset_index(inplace=True)
stadt_PM2_5_average['datum'] = pd.to_datetime(stadt_PM2_5_average['datum'])
stadt_PM2_5_average['datum_numeric'] = (stadt_PM2_5_average['datum'] - stadt_PM2_5_average['datum'].min()).dt.days

# Lineare Regression
X = stadt_PM2_5_average[['datum_numeric']]
y = stadt_PM2_5_average['average_stadtpm2_5']
model = LinearRegression()
model.fit(X, y)
stadt_PM2_5_average['stadt_PM2_5_average_trend'] = model.predict(X)

In [236]:
# Berechnung von R^2 und MSE und MAE und die Steigung
r_squared_spm2_5_regression = r2_score(y, stadt_PM2_5_average['stadt_PM2_5_average_trend'])
mse_spm2_5_regression = mean_squared_error(y, stadt_PM2_5_average['stadt_PM2_5_average_trend'])
mae_spm2_5_regression = mean_absolute_error(y, stadt_PM2_5_average['stadt_PM2_5_average_trend'])
slope_spm2_5 = model.coef_[0]

# Visualisierung mit Plotly Express
fig = px.scatter(stadt_PM2_5_average, x='datum', y='average_stadtpm2_5', title='Lineare Regression für average_stadtpm10')
fig.add_scatter(x=stadt_PM2_5_average['datum'], y=stadt_PM2_5_average['stadt_PM2_5_average_trend'], mode='lines', name='Trend')
fig.update_layout(xaxis_title='Datum', yaxis_title='average_stadtpm2_5')
fig.show()

In [237]:
print(r_squared_spm2_5_regression)
print(mae_spm2_5_regression)
print(slope_spm2_5)

0.0010935856143848843
3.415348841850595
0.0007351640664628209


Jetzt wird die Prognose für die Tabelle der PM10 Werte in der Stadt gemacht

es wird wieder die Tabelle eingelesen und in Trainings- und Testdaten unterteilt

In [238]:
# CSV-Datei einlesen
average_values = pd.read_csv("werte_daten/average_values.csv")
# es werden nur die nötigten Werte genommen
stadt_PM10_average = average_values[["datum", "average_stadtpm10"]]
# Zeile mit dem Datum 15. März 2023 entfernen
stadt_PM10_average = stadt_PM10_average[stadt_PM10_average['datum'] != '2023-03-15']
# Das Datum als Index setzen
stadt_PM10_average.set_index('datum', inplace=True)

# ein Series mit den Durchschnittsdaten erstellen
values_spm10 = stadt_PM10_average["average_stadtpm10"]
# das filtern der Datumswerte für die Visualisierung später
zeitreihen_index_spm10 = values_spm10.index

In [239]:
# die Funktion ist dafür da, um die Values in X und y Werte zu unterteilen, mit einer gegebenen window-size
def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+5]]
        X.append(row)
        label = df_as_np[i+5]
        y.append(label)
    return np.array(X), np.array(y)

WINDOW_SIZE = 5
X, y = df_to_X_y(values_spm10, WINDOW_SIZE)
X.shape, y.shape

((368, 5, 1), (368,))

In [240]:
# die Daten werden in Traings-, Testdaten unterteilt, zudem wird auch der Index unterteilt
X_train, y_train = X[:280], y[:280]
train_index = zeitreihen_index_spm10[5:285]
X_val, y_val = X[:300], y[:300]
val_index = zeitreihen_index_spm10[5:305]
X_test, y_test = X[300:], y[300:]
test_index = zeitreihen_index_spm10[305:]

es wird wieder das Modell trainiert

In [165]:
# es wird ein Modell erstellt, trainiert und gespeichert
model4 = Sequential()
model4.add(InputLayer((5,1)))
model4.add(LSTM(64))
model4.add(Dense(8, 'relu'))
model4.add(Dense(1, 'linear'))
model4.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()])
model4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)
model4.save('model4.keras')

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 164.0506 - root_mean_squared_error: 12.6608 - val_loss: 120.4390 - val_root_mean_squared_error: 10.9745
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 105.6682 - root_mean_squared_error: 10.2731 - val_loss: 88.2535 - val_root_mean_squared_error: 9.3943
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 64.4157 - root_mean_squared_error: 7.9708 - val_loss: 70.0449 - val_root_mean_squared_error: 8.3693
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 78.3316 - root_mean_squared_error: 8.7298 - val_loss: 64.7530 - val_root_mean_squared_error: 8.0469
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 50.8752 - root_mean_squared_error: 7.1226 - val_loss: 62.9698 - val_root_mean_squared_error: 7.9353
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 61.7346 - root_mean_squared_error: 7.8338 - val_loss: 60.9160 - val_root_mean_squared_error: 7.8049
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - 

In [241]:
# das Laden des Modells
model4 = load_model('model4.keras')

In [242]:
# es werden die Traingsdaten vorhergesagt und mit den tatsächlichen Werten in ein df geschrieben
train_predictions = model4.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions' : train_predictions, 'Actuals' : y_train}, index=train_index)
from sklearn.metrics import mean_absolute_error
# Berechnung des Mean Absolute Error (MAE) und R^2 und des MSE
mae_spm10 = mean_absolute_error(train_results['Actuals'], train_results['Train Predictions'])
r_squared_spm10 = r2_score(train_results['Actuals'], train_results['Train Predictions'])
mse_spm10 = mean_squared_error(train_results['Actuals'], train_results['Train Predictions'])

print(f'Mean Absolute Error: {mae_spm10}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Mean Absolute Error: 3.7508526289326807


es werden wieder Zukunftsvorhersagen gemacht

In [148]:
# die Eingabedaten für die Vorhersage der Zukunft werden erstellt
X_next = X_test  
predictions = []

for _ in range(5):
    y_pred = model1.predict(X_next)
    predictions.append(y_pred)
    # Aktualisieren von X_next für den nächsten Schritt
    X_next = update_X_next(X_next, y_pred)  

# Konvertiere new_index in einen DatetimeIndex
new_index_dt = pd.to_datetime(new_index)

# Kombiniere beide Indexstrukturen
gesamt_index = zeitreihen_index_spm10.append(new_index_dt)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  


In [171]:
# Vorhersage für den ersten Schritt machen
y_pred_step1 = model4.predict(X_test)

# Vorbereiten der Daten für Plotly Express
historical_data = values_spm10

# Plotly Express-Figur erstellen
fig = px.line(title='Vorhersage für den ersten Schritt mit LSTM-Modell')

# Plot der historischen Daten
fig.add_scatter(x=gesamt_index[:len(historical_data)], y=historical_data, mode='lines+markers', name='Historische Daten')

# Plot der Vorhersage für den ersten Schritt
x_values_pred = gesamt_index[len(historical_data):len(historical_data) + len(y_pred_step1)]
fig.add_scatter(x=x_values_pred, y=y_pred_step1.flatten(), mode='lines+markers', name='Vorhersage')

fig.update_layout(xaxis_title='Zeit', yaxis_title='Wert', title='Vorhersage für den Wert PM2.5 in der Stadt mit LSTM-Modell')
fig.show()

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


Nun wird eine Lineare Regression versucht

In [243]:
# Datum in numerisches Format umwandeln
stadt_PM10_average = stadt_PM10_average.copy()
stadt_PM10_average.reset_index(inplace=True)
stadt_PM10_average['datum'] = pd.to_datetime(stadt_PM10_average['datum'])
stadt_PM10_average['datum_numeric'] = (stadt_PM10_average['datum'] - stadt_PM10_average['datum'].min()).dt.days

# Lineare Regression
X = stadt_PM10_average[['datum_numeric']]
y = stadt_PM10_average['average_stadtpm10']
model = LinearRegression()
model.fit(X, y)
stadt_PM10_average['average_stadtpm10_trend'] = model.predict(X)

In [244]:
# Berechnung von R^2 und MSE und MAE und die Steigung
r_squared_spm10_regression = r2_score(y, stadt_PM10_average['average_stadtpm10_trend'])
mse_spm10_regression = mean_squared_error(y, stadt_PM10_average['average_stadtpm10_trend'])
mae_spm10_regression = mean_absolute_error(y, stadt_PM10_average['average_stadtpm10_trend'])
slope_spm10 = model.coef_[0]

# Visualisierung mit Plotly Express
fig = px.scatter(stadt_PM10_average, x='datum', y='average_stadtpm10', title='Lineare Regression für average_stadtpm10')
fig.add_scatter(x=stadt_PM10_average['datum'], y=stadt_PM10_average['average_stadtpm10_trend'], mode='lines', name='Trend')
fig.update_layout(xaxis_title='Datum', yaxis_title='average_stadtpm10')
fig.show()

In [245]:
print(r_squared_spm10_regression)
print(mae_spm10_regression)
print(slope_spm10)

0.0029282588843474455
5.444560977284486
0.00198784965608942
